# Price prediction and Classification Notebook
This notebook loads data and prepares  for price prediction workflow.

In [ ]:
import pandas as pd
import numpy as py

# Load CSV file
df = pd.read_csv(r"C:/Users/DELL/Documents/NLP/wfp_food_prices_eth.csv")

# Access a column
print(df.shape)
print(df.columns)

# View first 5 rows
print(df.head(8))

In [ ]:
import pandas as pd
import numpy as np

# Drop first row, reset index, convert date column to datetime, and cast numeric columns
df = (
    df.drop(0)
      .reset_index(drop=True)
      .assign(date=lambda x: pd.to_datetime(x['date']))
      .astype({'price': float, 'usdprice': float})
)

# Sort by date, drop duplicates, reset index
df = df.sort_values('date').drop_duplicates().reset_index(drop=True)

# Extract year and month
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

# Clean commodity column: remove parentheses text and extract variety
df['commodity_base'] = df['commodity'].str.replace(r'\s*\(.*?\)', '', regex=True).str.strip()
df['variety'] = df['commodity'].str.extract(r'\((.*?)\)', expand=False).fillna('')

# Check missing values
print(df.isnull().sum())

# Replace unavailable admin2 values with NaN
df['admin2'] = df['admin2'].replace('Administrative unit not available', np.nan)

# Convert latitude and longitude to numeric
df[['latitude', 'longitude']] = df[['latitude', 'longitude']].apply(pd.to_numeric, errors='coerce')

# Keep only rows with positive price
df = df[df['price'] > 0].reset_index(drop=True)

# Convert categorical columns
df[['admin1', 'admin2', 'market', 'pricetype', 'category']] = df[['admin1', 'admin2', 'market', 'pricetype', 'category']].astype('category')

# Handle missing admin2 values
df['admin2'] = df['admin2'].cat.add_categories('Unknown').fillna('Unknown')

# Drop rows with missing admin1, latitude, longitude
df = df.dropna(subset=['admin1', 'latitude', 'longitude']).reset_index(drop=True)

# Replace empty variety with 'Standard'
df['variety'] = df['variety'].replace('', 'Standard').fillna('Standard')

# Clean unit column
df['unit'] = df['unit'].str.strip().str.upper()

# Drop unnecessary columns
df = df.drop(columns=['commodity','priceflag','currency'])

# Final checks
print(df.shape)
print(df.tail())
print(df.head())

In [ ]:
encode_cols = [
    'admin1','admin2','market',
    'category','pricetype',
    'unit','commodity_base','variety'
]

# Show unique counts and sample categories
for col in encode_cols:
    print(f"\nColumn: {col}")
    print(f"Unique count: {df[col].nunique()}")
    print(f"Categories: {df[col].unique()[:10]}")  # show first 10 unique values

In [ ]:
print(df.columns)
print(df.info())
df.describe()

In [ ]:

# ============================================================
# CORRELATION HEATMAP
# ============================================================

import matplotlib.pyplot as plt
import seaborn as sns

# Use only numeric columns for correlation
numeric_df = df.select_dtypes(include=['int64','float64'])

# Compute correlation matrix
corr_matrix = numeric_df.corr()

# Set up the matplotlib figure
plt.figure(figsize=(14,10))

# Draw the heatmap with annotations
sns.heatmap(
    corr_matrix,
    annot=True,        # show correlation values
    fmt=".2f",         # 2 decimal points
    cmap='coolwarm',   # color map
    cbar=True,
    square=True,
    linewidths=0.5
)

plt.title("Correlation Heatmap of Features", fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# FULL DATA PREPROCESSING PIPELINE (ML READY)
# ============================================================

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt

# -----------------------------
# 1️⃣ CLEAN & STANDARDIZE UNITS
# -----------------------------
df['unit'] = df['unit'].astype(str).str.strip().str.upper()

# Convert bulk to per KG (optional but recommended)
df.loc[df['unit']=='100 KG','usdprice'] /= 100
df['unit'] = df['unit'].replace('100 KG','KG')

# -----------------------------
# 2️⃣ CLEAN & FIX PRICETYPE
# -----------------------------
df['pricetype'] = df['pricetype'].astype(str).str.strip().str.lower()

# Map to numeric safely using map (avoids FutureWarning)
pricetype_map = {'retail': 0, 'wholesale': 1, '0': 0, '1': 1}
df['pricetype'] = df['pricetype'].map(pricetype_map)

# Fill any remaining NaNs with mode
df['pricetype'] = df['pricetype'].fillna(df['pricetype'].mode()[0]).astype(int)

# -----------------------------
# 3️⃣ CLEAN & CONVERT OTHER CATEGORICALS
# -----------------------------
cat_cols = ['admin1','admin2','market','category']
for col in cat_cols:
    df[col] = df[col].astype(str).str.strip().replace('', 'Unknown').astype('category')

# -----------------------------
# 4️⃣ ONE-HOT ENCODING (LOW CARDINALITY)
# -----------------------------
if 'category' in df.columns:
    df = pd.get_dummies(df, columns=['category'], drop_first=True)

# -----------------------------
# 5️⃣ LABEL ENCODING (HIGH CARDINALITY)
# -----------------------------
label_cols = ['admin1','admin2','market','unit','commodity_base','variety']
encoders = {}
for col in label_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    encoders[col] = le

# -----------------------------
# 6️⃣ FINAL CHECK
# -----------------------------
print("✅ Preprocessing Complete")
print("Shape:", df.shape)
print("\nData Types:\n", df.dtypes)
print("\nSample Data:")
print(df.head())

# -----------------------------
# 7️⃣ CORRELATION HEATMAP
# -----------------------------
numeric_df = df.select_dtypes(include=['int64','float64'])
plt.figure(figsize=(12,10))
sns.heatmap(numeric_df.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Heatmap (Numeric Features)")
plt.show()